In [274]:
import pandas as pd, docx2txt
from io import StringIO
import os
import re

In [275]:
#read .docx into jupyter
text = docx2txt.process ("C://Users/Bryant/Documents/GitHub/ACT/archive/DataPrep/ACT Math Sheets_v2_3_9_18/Geometry Rules.docx")

In [276]:
text

'Geometry Rules\n\nApril 2017\n\n7. In parallelogram ABCD below AC(line) is a diagonal, the measure of (angle)ABC is 40°, and the measure of (angle)ACD is 57°. What is the measure of (angle)CAD?\n\n40°\n\n57°\n\n77°\n\n83°\n\n97°\n\n[PICTURE]\n\n\n\n60. Ray (ray)PK bisects (angle)LPM, the measure of (angle)LPM is 11x°, and the measure of (angle)LPK is (4x + 18)°. What is the measure of (angle)KPM?\n\n12°\n\n28 (2/7)°\n\n42°\n\n61 (1/5)°\n\n66°\n\n\n\nJune 2017\n\n6. in triangle(ACD) below, B is on line(AC), E is on line(AD), the measure of angle(CAD) is 28°, and line(AD) is perpendicular to both line(BE) and line(CD). What is the measure of angle(CBE)?\n\n104°\n\n118°\n\n124°\n\n146°\n\n152°\n\n[PICTURE]\n\n\n\n13. In the figure shown below, E and G lie on (line)AC, D and F lie on (line)AB , (line)DE and (line)FG are parallel to the (line)BC, and the given lengths are in feet. What is the length of (line)AC, in feet? \n\n9\n\n18\n\n21\n\n30\n\n36\n\n[PICTURE]\n\n\n\n46. in the figure b

## Clean text. Each question will need the following categories:

#### - ID (to be used as a Primary Key in MySQL db)
#### - Test Date
#### - Question Number
#### - Question Statement
#### - Answer Choices (one choice for each column, 5 total)
#### - Picture (if necessary)

In [277]:
#split text into list separated by \n\n i.e. 'enter' button
text_split = [line for line in StringIO(text).read().split('\n\n')]

In [278]:
#remove empty entries and spaces
try:
    text_split.remove('')
    text_split.remove(' ')
except:
    pass

In [279]:
text_split

['Geometry Rules',
 'April 2017',
 '7. In parallelogram ABCD below AC(line) is a diagonal, the measure of (angle)ABC is 40°, and the measure of (angle)ACD is 57°. What is the measure of (angle)CAD?',
 '40°',
 '57°',
 '77°',
 '83°',
 '97°',
 '[PICTURE]',
 '60. Ray (ray)PK bisects (angle)LPM, the measure of (angle)LPM is 11x°, and the measure of (angle)LPK is (4x + 18)°. What is the measure of (angle)KPM?',
 '12°',
 '28 (2/7)°',
 '42°',
 '61 (1/5)°',
 '66°',
 '',
 'June 2017',
 '6. in triangle(ACD) below, B is on line(AC), E is on line(AD), the measure of angle(CAD) is 28°, and line(AD) is perpendicular to both line(BE) and line(CD). What is the measure of angle(CBE)?',
 '104°',
 '118°',
 '124°',
 '146°',
 '152°',
 '[PICTURE]',
 '',
 '13. In the figure shown below, E and G lie on (line)AC, D and F lie on (line)AB , (line)DE and (line)FG are parallel to the (line)BC, and the given lengths are in feet. What is the length of (line)AC, in feet? ',
 '9',
 '18',
 '21',
 '30',
 '36',
 '[PICTURE

In [280]:
#store topic
topic = text_split[0]
full_list = []
new = []
for line in text_split[1:]:
    #store month and year of test date
    if re.search('(^December|June|April|July)', line):
        month = re.search('(^December|June|April|July)', line).group()
        year = re.search('$2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020',line).group()
        #continue to next iteration if test date is found
        continue
        
    #look for question number to initiate a new row entry
    #len(line) > 10 to avoid finding answers that begin with numbers
    if re.search('^..\.', line) and len(line) > 10:
        #append existing list to full_list, then reinitiate 'new' row
        if len(new) > 0:
            full_list.append(new)
            new = []
        #append various data for 'new' row entry 
        num = re.search('^..\.', line).group().replace(".","")
        #id column
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        #question statement
        new.append(line)
        continue

    #same as previous block with exception re.search looking for single digits
    if re.search('^.\.', line) and len(line) > 10:
        if len(new) > 0:
            full_list.append(new)
            new = []
        #store question  number    
        num = re.search('^.\.', line).group().replace(".","")
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        new.append(line)
        continue
    #only lines that reach 'else' statement should be answer choices; append to list   
    else:
        new.append(line)

In [281]:
full_list

[['Apr.2017.7',
  'Geometry Rules',
  'April.2017',
  '7',
  '7. In parallelogram ABCD below AC(line) is a diagonal, the measure of (angle)ABC is 40°, and the measure of (angle)ACD is 57°. What is the measure of (angle)CAD?',
  '40°',
  '57°',
  '77°',
  '83°',
  '97°',
  '[PICTURE]'],
 ['Apr.2017.60',
  'Geometry Rules',
  'April.2017',
  '60',
  '60. Ray (ray)PK bisects (angle)LPM, the measure of (angle)LPM is 11x°, and the measure of (angle)LPK is (4x + 18)°. What is the measure of (angle)KPM?',
  '12°',
  '28 (2/7)°',
  '42°',
  '61 (1/5)°',
  '66°',
  ''],
 ['Jun.2017.6',
  'Geometry Rules',
  'June.2017',
  '6',
  '6. in triangle(ACD) below, B is on line(AC), E is on line(AD), the measure of angle(CAD) is 28°, and line(AD) is perpendicular to both line(BE) and line(CD). What is the measure of angle(CBE)?',
  '104°',
  '118°',
  '124°',
  '146°',
  '152°',
  '[PICTURE]',
  ''],
 ['Jun.2017.13',
  'Geometry Rules',
  'June.2017',
  '13',
  '13. In the figure shown below, E and G li

## ready file for import into MySQL db

In [282]:
#prep for import into MySQL db

from pandas import DataFrame
#convert list to df
df = DataFrame.from_records(full_list)
#export to .csv excluding index and header
df.to_csv('GeometryRules.csv', index=False, header=False)